In [0]:
# !! pip install arff

['Collecting arff',
 '  Downloading https://files.pythonhosted.org/packages/50/de/62d4446c5a6e459052c2f2d9490c370ddb6abc0766547b4cef585913598d/arff-0.9.tar.gz',
 'Building wheels for collected packages: arff',
 '  Building wheel for arff (setup.py) ... \x1b[?25l\x1b[?25hdone',
 '  Stored in directory: /root/.cache/pip/wheels/04/d0/70/2c73afedd3ac25c6085b528742c69b9587cbdfa67e5194583b',
 'Successfully built arff',
 'Installing collected packages: arff',
 'Successfully installed arff-0.9']

In [1]:
import arff
import time
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from operator import attrgetter, itemgetter
from io import StringIO
from sklearn.model_selection import train_test_split
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix,classification_report,precision_recall_fscore_support as score, average_precision_score
from sklearn import metrics
from sklearn.model_selection import cross_validate
from pickle import Pickler, Unpickler

## training model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

ModuleNotFoundError: No module named 'google'

In [ ]:
root_path = "gdrive/My Drive/Colab Notebooks/"
df = Unpickler(open(root_path + "dataset/OffComBR3.sav", 'rb')).load()

X = df['sentence'].tolist()
y = df['hate'].tolist()
X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, test_size=0.33, random_state=42)

In [ ]:
parameters_dt = {'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
                 'tfidf__max_df': [0.2, 0.6, 0.8, 1.0],
                 'tfidf__min_df': [1, 2, 5, 10],
                 'tfidf__use_idf': [True, False],
                 'tfidf__smooth_idf': [True, False],
                 'tfidf__sublinear_tf': [True, False],
                 'tfidf__strip_accents': ['ascii'],
                  'clf__criterion': ['gini'],
                  'clf__class_weight': [{0: 1, 1: 2}],
                  'clf__min_samples_split': [3]
                 }

parameters_mlp = {'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
                 'tfidf__max_df': [0.2, 0.6, 0.8, 1.0],
                 'tfidf__min_df': [1, 2, 5, 10],
                 'tfidf__use_idf': [True, False],
                 'tfidf__smooth_idf': [True, False],
                 'tfidf__sublinear_tf': [True, False],
                 'tfidf__strip_accents': ['ascii'],
                 'clf__activation': ['logistic'],
                 'clf__solver': ['lbfgs'],
                 'clf__alpha': [0]
                }

parameters_mnb = {'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
                 'tfidf__max_df': [0.2, 0.6, 0.8, 1.0],
                 'tfidf__min_df': [1, 2, 5, 10],
                 'tfidf__use_idf': [True, False],
                 'tfidf__smooth_idf': [True, False],
                 'tfidf__sublinear_tf': [True, False],
                 'tfidf__strip_accents': ['ascii'],
                'clf__alpha': [0.1, 0.2],
                'clf__fit_prior': [False],
                }

parameters_rf = {'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
                 'tfidf__max_df': [0.2, 0.6, 0.8, 1.0],
                 'tfidf__min_df': [1, 2, 5, 10],
                 'tfidf__use_idf': [True, False],
                 'tfidf__smooth_idf': [True, False],
                 'tfidf__sublinear_tf': [True, False],
                 'tfidf__strip_accents': ['ascii'],
                 'clf__n_estimators': [100],
                 'clf__max_depth': [None],
                 'clf__min_samples_split': [2, 4],
                 'clf__min_samples_leaf': [1, 2, 4]
                }

parameters_sgd = {'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
                 'tfidf__max_df': [0.2, 0.6, 0.8, 1.0],
                 'tfidf__min_df': [1, 2, 5, 10],
                 'tfidf__use_idf': [True, False],
                 'tfidf__smooth_idf': [True, False],
                 'tfidf__sublinear_tf': [True, False],
                 'tfidf__strip_accents': ['ascii'],
                 'clf__alpha': [0.01],
                 'clf__loss': ['perceptron'],
                 'clf__penalty': ['none']
                 }


parameters_svc = {'tfidf__ngram_range': [(1,1), (1,2), (1,3)],
                 'tfidf__max_df': [0.2, 0.6, 0.8, 1.0],
                 'tfidf__min_df': [1, 2, 5, 10],
                 'tfidf__use_idf': [True, False],
                 'tfidf__smooth_idf': [True, False],
                 'tfidf__sublinear_tf': [True, False],
                 'tfidf__strip_accents': ['ascii'],
                 'clf__C': [4],
                 'clf__kernel': ['linear'],
                 'clf__shrinking': [True],
                 'clf__probability': [True],
                 'clf__tol': [1]
                 }

parameters = [parameters_dt, parameters_mlp, parameters_mnb, parameters_rf, parameters_sgd, parameters_svc]

In [ ]:
classifiers = [DecisionTreeClassifier(), MLPClassifier(), MultinomialNB(), RandomForestClassifier(), SGDClassifier(), SVC()]

In [ ]:
def get_recal_1(y_true, y_pred):
    precision, recall, fscore, support = score(y_true, y_pred)
#     print(classification_report(y_true, y_pred))
    return(recall[1])

In [3]:
for i, classifier in enumerate(classifiers):
  cl = Pipeline([
          ('tfidf',TfidfVectorizer()),
          ('clf', classifier),
        ])

  gs_clf = GridSearchCV(cl, parameters[i], cv=2, iid=False, n_jobs=-1, scoring=make_scorer(get_recal_1))

  gs_clf.fit(X, y)
  print(classifier.__class__.__name__)
  print(gs_clf.best_score_)
  print(gs_clf.best_params_)

NameError: name 'classifiers' is not defined

In [4]:
## classificadores com os melhores hiperparametros

dt =  Pipeline([
        ('tfidf',TfidfVectorizer(max_df = 0.6,
                                 min_df = 1,
                                 ngram_range = (1, 3),
                                 smooth_idf = True,
                                 strip_accents = 'ascii',
                                 sublinear_tf = False,
                                 use_idf = True)),
        ('clf', DecisionTreeClassifier(class_weight= {0: 1, 1: 2}, criterion= 'gini', min_samples_split = 3)),
        ])


mlp =  Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 5,
                                 ngram_range= (1, 2),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= False,
                                 use_idf = False)),
        ('clf', MLPClassifier(activation='logistic', alpha= 0, solver='lbfgs')),
        ])

mnb =  Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.6,
                                 min_df= 5,
                                 ngram_range= (1, 1),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', MultinomialNB(alpha=0.1, fit_prior=False)),
        ])

rf = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', RandomForestClassifier(max_depth=None, min_samples_leaf= 1, min_samples_split= 2, n_estimators=100)),
        ])


sgd = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 10,
                                 ngram_range= (1, 3),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = False)),
        ('clf', SGDClassifier(alpha=0.01, loss='perceptron', penalty='none')),
        ])


svc =  Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.6,
                                 min_df= 1,
                                 ngram_range= (1, 1),
                                 smooth_idf= True,
                                 strip_accents= 'ascii',
                                 sublinear_tf= False,
                                 use_idf = True)),
        ('clf', SVC(C=4, kernel ='linear', probability=True, shrinking=True, tol=1)),
        ])

all_improved_classifiers = [('dt', dt), ('mlp', mlp), ('mnb', mnb), ('rf', rf), ('sgd', sgd), ('svc', svc)]
soft_improved_classifiers = [('mlp', mlp), ('mnb', mnb), ('rf', rf), ('svc', svc)]

In [6]:
for classifier in improved_classifiers:
    classifier.fit(X_train, y_train)

    print(classifier['clf'].__class__.__name__)
    ## Report normal
    pred = classifier.predict(X_test)
    print(classification_report(y_test, pred))

    ## Métricas do CV:

    recall_1 = cross_val_score(classifier, X, y, cv=10, scoring= make_scorer(get_recal_1))
    print("Recall 1: %0.2f (+/- %0.2f) [%s]" % (recall_1.mean(), recall_1.std(), classifier['clf'].__class__.__name__))
    recall = cross_val_score(classifier, X, y, cv=10, scoring='recall')
    print("Recall: %0.2f (+/- %0.2f) [%s]" % (recall.mean(), recall.std(), classifier['clf'].__class__.__name__))
    accuracy = cross_val_score(classifier, X, y, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (accuracy.mean(), accuracy.std(), classifier['clf'].__class__.__name__))
    precision = cross_val_score(classifier, X, y, cv=10, scoring='precision')
    print("Precision: %0.2f (+/- %0.2f) [%s]" % (precision.mean(), precision.std(), classifier['clf'].__class__.__name__))
    print("=================================================================")
  

NameError: name 'improved_classifiers' is not defined

In [0]:
## classificadores com os melhores hiperparametros sem o tfidf

# dt =  DecisionTreeClassifier(class_weight= {0: 1, 1: 2}, criterion= 'gini', min_samples_split = 3)
# mlp = MLPClassifier(activation='logistic', alpha= 0, solver='lbfgs')
# mnb =  MultinomialNB(alpha=0.1, fit_prior=False)
# rf = RandomForestClassifier(max_depth=None, min_samples_leaf= 1, min_samples_split= 2, n_estimators=100)
# sgd = SGDClassifier(alpha=0.01, loss='perceptron', penalty='none')
# svc = SVC(C=4, kernel ='linear', probability=True, shrinking=True, tol=1)

# only_classifiers = [dt, mlp, mnb, rf, sgd, svc]

In [0]:
### TESTANDO APENAS VOTING = HARD
voting = Pipeline([
    ('voting', VotingClassifier(estimators = all_improved_classifiers))
    
])


def combinations_on_off(num_classifiers):
    return [[int(x) for x in list("{0:0b}".format(i).zfill(num_classifiers))]
            for i in range(1, 2 ** num_classifiers)]


parameters_voting = {
                      'voting__voting': ['hard'],
                      'voting__weights': combinations_on_off(len(all_improved_classifiers))
                }


gridVoting = GridSearchCV(voting, parameters_voting, cv=10, iid=False, n_jobs=-1, scoring=make_scorer(get_recal_1))
gridVoting.fit(X, y)
print(gridVoting.best_score_)
print(gridVoting.best_params_)

0.5497619047619048
{'voting__voting': 'hard', 'voting__weights': [0, 0, 1, 0, 0, 0]}


In [0]:
### TESTANDO APENAS VOTING = SOFT
voting = Pipeline([
    ('voting', VotingClassifier(estimators = soft_improved_classifiers))
    
])


parameters_voting = {
                      'voting__voting': ['soft'],
                      'voting__weights': combinations_on_off(len(soft_improved_classifiers))
                }


gridVoting2 = GridSearchCV(voting, parameters_voting, cv=10, iid=False, n_jobs=-1, scoring='recall')
gridVoting2.fit(X, y)
print(gridVoting2.best_score_)
print(gridVoting2.best_params_)

0.5497619047619048
{'voting__voting': 'soft', 'voting__weights': [0, 1, 0, 0]}


In [0]:
### TESTANDO APENAS VOTING = SOFT
voting = Pipeline([
    ('voting', VotingClassifier(estimators = soft_improved_classifiers))
    
])


parameters_voting = {
                      'voting__voting': ['soft'],
                      'voting__weights': combinations_on_off(len(soft_improved_classifiers))
                }


gridVoting3 = GridSearchCV(voting, parameters_voting, cv=10, iid=False, n_jobs=-1, scoring='accuracy')
gridVoting3.fit(X, y)
print(gridVoting3.best_score_)
print(gridVoting3.best_params_)

0.8208985916995625
{'voting__voting': 'soft', 'voting__weights': [0, 0, 1, 1]}


In [0]:
### TESTANDO APENAS VOTING = SOFT
voting = Pipeline([
    ('voting', VotingClassifier(estimators = soft_improved_classifiers))
    
])


parameters_voting = {
                      'voting__voting': ['soft'],
                      'voting__weights': combinations_on_off(len(soft_improved_classifiers))
                }


gridVoting3 = GridSearchCV(voting, parameters_voting, cv=10, iid=False, n_jobs=-1, scoring='accuracy')
gridVoting3.fit(X, y)
print(gridVoting3.best_score_)
print(gridVoting3.best_params_)

0.6470183982683982
{'voting__voting': 'soft', 'voting__weights': [0, 0, 1, 1]}


In [0]:
voting = Pipeline([
    ('voting', VotingClassifier(estimators = [('mnb', mnb), ('mlp', mlp), ('svc', svc)]))
    
])


parameters_voting = {
                      'voting__voting': ['soft'],
                      'voting__weights': [[0,0,1],
                                         [0,1,0],
                                         [1,0,0],
                                         [1,1,1],
                                         [1,1,2],
                                         [1,2,1],
                                         [2,1,1],
                                         [2,2,1],
                                         [1,2,2]]
                }


gridVoting3 = GridSearchCV(voting, parameters_voting, cv=10, iid=False, n_jobs=-1, scoring='recall')
gridVoting3.fit(X, y)
print(gridVoting3.best_score_)
print(gridVoting3.best_params_)

0.5497619047619048
{'voting__voting': 'soft', 'voting__weights': [1, 0, 0]}


In [0]:
  mnb.fit(X_train, y_train)
  rf.fit(X_train, y_train)
  sgd.fit(X_train, y_train)
  svc.fit(X_train, y_train)
  dt.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.6, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents='ascii',
                                 sublinear_tf=False,
                                 token_patter...
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 DecisionTreeClassifier(class_weight={0: 1, 1: 2},
                                        criterion='gini', max_depth=None,
                      

In [0]:
def classifica(txt):
  result = []
  for t in txt:
    if (mnb.predict([t]) == 1) or (rf.predict([t]) == 1) or (svc.predict([t]) == 1):
      result.append(1)
    else:
      result.append(0)
  return result

In [0]:
pred = classifica(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.87      0.77      0.82       267
           1       0.41      0.57      0.47        74

    accuracy                           0.73       341
   macro avg       0.64      0.67      0.65       341
weighted avg       0.77      0.73      0.74       341

